In [1]:
import pygrib
import numpy as np
import pickle
import os
from collections import OrderedDict
import zipfile

In [29]:
Time = []
try:
    os.mkdir('US_wind')
except:
    pass

for rawfname in os.listdir(os.getcwd() + '/raw'):
    print(rawfname)
    grbs = pygrib.open(os.getcwd() + '/raw/' + rawfname)
    uin = grbs.select(shortName='u', typeOfLevel='isobaricInhPa', level = lambda l: l >= 100 and l <= 1000)
    vin = grbs.select(shortName='v', typeOfLevel='isobaricInhPa', level = lambda l: l >= 100 and l <= 1000)
    grbs.close()

    for i in range(len(uin)):
        u_wind_i = uin[i]
        v_wind_i = vin[i]

        lat_grid = u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[1].flatten()
        lon_grid = u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[2].flatten()
        lon_lat_grid = np.dstack((lon_grid, lat_grid))[0]

        pickle.dump(lon_lat_grid, open('LonLat_Grid.p','wb'), protocol=2)

        date_time = str(u_wind_i.year) + '_' + str(u_wind_i.month).zfill(2) + '_' + str(u_wind_i.day).zfill(2) + '_' + str(u_wind_i.hour).zfill(2) + '00'
        if date_time not in Time:

            wind_data = OrderedDict()
            Time.append(date_time)

            wind_data[u_wind_i.level] = np.dstack((u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten(),
                                                     v_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten()))[0]

        else:
            wind_data[u_wind_i.level] = np.dstack((u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten(),
                                                     v_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten()))[0]

        pickle.dump(wind_data, open(os.getcwd() + '/US_wind/' + date_time + '.p','wb'), protocol=2) # m/s

A24522-201310.pgb.f00
A24351-201301.pgb.f00
A24560-201312.pgb.f00
A24370-201302.pgb.f00
A24389-201303.pgb.f00
A24446-201306.pgb.f00
A24541-201311.pgb.f00
A24503-201309.pgb.f00
A24408-201304.pgb.f00
A24484-201308.pgb.f00
A24465-201307.pgb.f00
A24427-201305.pgb.f00


In [31]:
with zipfile.ZipFile('NCAR_wind.zip', mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
    for fname in os.listdir(os.getcwd() + '/US_wind/'):
        zf.write(os.getcwd() + '/US_wind/' + fname)